In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

# Current directory
print(os.getcwd())

# change directory
os.chdir('/content/drive/MyDrive/python_training/NLP100Days/project_3_spam_detector_2')
print(os.getcwd())

/content
/content/drive/MyDrive/python_training/NLP100Days/project_3_spam_detector_2


In [3]:
from __future__ import print_function, division
from builtins import range

import pandas as pd
import numpy as np

In [4]:
# 註: 理論上 multinomial NB 是針對出現次數 "counts", 但文件上說對出現比率 "word proportions"也適合

#./sample_data/spambase.data
data = pd.read_csv('spambase.data').values # use pandas for convenience
np.random.shuffle(data) # shuffle each row in-place, but preserve the row

X = data[:,:48]
Y = data[:,-1]

# 不一定用100列 作測試 100->80 試試
Xtrain = X[:-80,]
Ytrain = Y[:-80,]
Xtest = X[-80:,]
Ytest = Y[-80:,]

In [5]:

# 我們在習題中，先試Naive Bayes
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(Xtrain, Ytrain)
print("Classification rate for NB:", model.score(Xtest, Ytest))

Classification rate for NB: 0.8875


In [6]:


# Decision Tree 的準確度如何？
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(Xtrain, Ytrain)
print("Classification rate for Decision Tree:", model.score(Xtest, Ytest))



Classification rate for Decision Tree: 0.9125


In [7]:
##### 任何 model都行，以下試試 AdaBoost! #####
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier()
model.fit(Xtrain, Ytrain)
print("Classification rate for AdaBoost:", model.score(Xtest, Ytest))

#####也可試試其他的
## https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html


Classification rate for AdaBoost: 0.875
